In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os 
from pprint import pprint

In [3]:
import numpy as np
import decimal
import copy

In [4]:
import time

In [5]:
import streamm

In this getting started example we will calculate the coupling between P3HT oligomers

In [6]:
import logging
logging.basicConfig(filename='p3ht_et.log',level=logging.DEBUG)

Now let's create project and resource to keep track of our work

In [7]:
p3ht_et = streamm.Project('P3HT_ET')

And a resource object to keep track of where our files are 

In [8]:
res_local = streamm.Resource('local')

Update relative location of templates directory 

In [9]:
res_local.dir['templates'] =  os.path.join(res_local.dir['home'],'..','templates','')

In [10]:
res_local.make_dir()

In [11]:
p3ht_et.set_resource(res_local)

Now we need to set a remote resource we have ssh access to and run calculations 

In [12]:
peregrine = streamm.Resource('peregrine')

In [13]:
peregrine.meta['type'] = "ssh"
peregrine.ssh['username'] = "tkemper"    
peregrine.ssh['address'] = "peregrine.hpc.nrel.gov"    
peregrine.dir['storage'] = '/mss/users/%s'%(peregrine.ssh['username'])
peregrine.dir['scratch'] = '/scratch/%s'%(peregrine.ssh['username'])
peregrine.dir['home'] = res_local.dir['home']
peregrine.dir['launch'] = res_local.dir['launch']
peregrine.dir['templates'] = res_local.dir['templates']

# Set default simulation specs
peregrine.properties['allocation'] = 'orgopv'
peregrine.properties['e-mail'] = 'travis.kemper@nrel.gov'
peregrine.properties['scratch'] = peregrine.dir['scratch']
peregrine.properties['allocation'] = 'orgopv'
peregrine.properties['walltime'] = 4
peregrine.properties['nodes'] = int(1)
peregrine.properties['ppn'] = int(24)
peregrine.properties['nproc'] = peregrine.properties['nodes']*peregrine.properties['ppn']
peregrine.properties['queue'] = 'short'
peregrine.properties['feature'] = '24core'
peregrine.properties['exe_command'] = 'qsub '

In [14]:
pprint(peregrine.properties)

{u'allocation': 'orgopv',
 'e-mail': 'travis.kemper@nrel.gov',
 u'exe_command': 'qsub ',
 u'feature': '24core',
 u'nodes': 1,
 u'nproc': 24,
 u'pmem': 1500,
 u'ppn': 24,
 u'queue': 'short',
 'scratch': '/scratch/tkemper',
 u'walltime': 4}


In [15]:
peregrine.make_dir()

Explicitely create a thiophene molecule

In [16]:
bbTh = streamm.Buildingblock('thiophene')
symbols = ['C','C','C','C','S','H','H','H','H']
positions = [ ]
positions.append([-1.55498576,-1.91131218,-0.00081000])
positions.append([-0.17775976,-1.91131218,-0.00081000])
positions.append([0.34761524,-0.57904218,-0.00081000])
positions.append([-0.65884476,0.36101082,0.00000000])
positions.append([-2.16948076,-0.35614618,-0.00000800])
positions.append([-2.18966076,-2.79526518,-0.00132100])
positions.append([0.45389024,-2.80145418,-0.00106400])
positions.append([1.41682424,-0.35961818,-0.00138200])
positions.append([-0.51943676,1.44024682,0.00064700])
for i in range(len(symbols)):
    pt_i = streamm.Particle(symbol=symbols[i])
    pos_i = positions[i]
    bbTh.add_partpos(pt_i,pos_i)

In [17]:
bbTh.particles[5].rsite = 'termcap'
bbTh.particles[6].rsite = 'funccap'
bbTh.particles[8].rsite = 'termcap'

Set some properties of the molecule to keep track of the parts

In [18]:
c_cnt =1
h_cnt =1

for pkey_i, particle_i  in bbTh.particles.iteritems():

    if( particle_i.symbol == 'C' ):
        particle_i.label = "C%d"%(c_cnt)
        particle_i.resname = "SCP2"
        particle_i.residue = 1

        c_cnt +=1 
    if( particle_i.symbol == 'S' ):
        particle_i.resname = "ThS"
        particle_i.residue = 2

    if( particle_i.symbol == 'H' ):
        particle_i.label = "H%d"%(h_cnt)
        particle_i.resname = "HA"
        particle_i.residue = 3

        h_cnt +=1 


Set the force-field type and guess some reasonable charges 

In [19]:
for pkey_i, particle_i  in bbTh.particles.iteritems():
    if( particle_i.symbol == 'C' ):
        particle_i.paramkey = 'CA'
        particle_i.charge = -0.025
    if( particle_i.symbol == 'S' ):
        particle_i.paramkey = 'S'
        particle_i.charge = -0.3
    if( particle_i.symbol == 'H' ):
        particle_i.paramkey = 'HA'
        particle_i.charge = 0.1

Check molecule is neutral 

In [20]:
total_charge = 0.0
for pkey_i, particle_i  in bbTh.particles.iteritems():
    total_charge += particle_i.charge
print total_charge

-2.77555756156e-17


Optimize structure with NWChem

But let's put it in a function this time

In [21]:
def nw_opt(project_i,bb_i,res_i):
    '''Optimize a streamm Buildingblock object with nwchem 
    
    '''
    nwchem_i = streamm.NWChem('nw_opt_{}'.format(bb_i.tag))
    print nwchem_i.tag 
    # Add thiophene structure 
    nwchem_i.strucC = copy.deepcopy(bb_i)
    # Set calculation to run on external resource
    nwchem_i.set_resource(res_i)
    # Make the local directories 
    nwchem_i.make_dir()
    #Change to the `launch` directory
    os.chdir(nwchem_i.dir['launch'])
    # Copy over templates
    nwchem_i.cp_file('templates','run',"nwchem_peregrine.pbs",'templates','launch')
    nwchem_i.cp_file('templates','nw',"nwchem.nw",'templates','launch')
    # Read in templates files 
    nwchem_i.load_str('templates','nw')        
    nwchem_i.load_str('templates','run')
    # Set calculation properties 
    nwchem_i.properties['basis'] = '6-31g'
    nwchem_i.properties['method'] = 'UHF'
    nwchem_i.properties['charge'] = 0
    nwchem_i.properties['spin_mult'] = 1
    nwchem_i.properties['task'] = 'SCF optimize'
    nwchem_i.properties['coord'] = nwchem_i.strucC.write_coord()
    # 
    pprint(nwchem_i.properties)
    # Replace <key> with properties value 
    nwchem_i.replacewrite_prop('nw','input','nw','%s.nw'%(nwchem_i.tag))
    nwchem_i.properties['input_nw'] = nwchem_i.files['input']['nw']
    nwchem_i.replacewrite_prop('run','scripts','run','%s.pbs'%(nwchem_i.tag))
    #
    nwchem_i.add_file('output','log',"%s.log"%(nwchem_i.tag))
    # Save details in .json files 
    os.chdir(nwchem_i.dir['home'])
    p3ht_et.dump_json()
    # 
    os.chdir(nwchem_i.dir['launch'])
    # 
    nwchem_i.push()
    # 
    nwchem_i.run()
    # Add calculation to project
    project_i.add_calc(nwchem_i,deepcopy = True)
    # 
    return project_i 

In [22]:
p3ht_et = nw_opt(p3ht_et,bbTh,peregrine)

nw_opt_thiophene
{u'allocation': 'orgopv',
 u'basis': '6-31g',
 u'charge': 0,
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 'coord': u'     C      -1.55498576      -1.91131218      -0.00081000 \n     C      -0.17775976      -1.91131218      -0.00081000 \n     C       0.34761524      -0.57904218      -0.00081000 \n     C      -0.65884476       0.36101082       0.00000000 \n     S      -2.16948076      -0.35614618      -0.00000800 \n     H      -2.18966076      -2.79526518      -0.00132100 \n     H       0.45389024      -2.80145418      -0.00106400 \n     H       1.41682424      -0.35961818      -0.00138200 \n     H      -0.51943676       1.44024682       0.00064700 \n',
 'e-mail': 'travis.kemper@nrel.gov',
 u'exe_command': 'qsub ',
 u'feature': '24core',
 u'finish_str': u'Total times  cpu:',
 u'maxiter': 100,
 u'method': 'UHF',
 u'nodes': 1,
 u'nproc': 24,
 u'pmem': 1500,
 u'ppn': 24,
 u'queue': 'short',
 'scratch': u'/scratch/tkemper/nw_opt_thiophene/'

In [ ]:
nwchem_i = p3ht_et.calculations['nw_opt_thiophene']

Check status unit finished

In [ ]:
while( nwchem_i.meta['status'] != 'finished'):
    nwchem_i.check()
    time.sleep(30)    

In [ ]:
print nwchem_i.meta['status']

In [ ]:
nwchem_i.analysis()

Print energies 

In [ ]:
print nwchem_i.properties['alpha_energies'][10:20]
print nwchem_i.properties['energy']

Check that the positions of the structure have been optimized 

In [ ]:
print bbTh.positions

In [ ]:
print nwchem_i.strucC.positions

Update positions with optimized geometry 

In [ ]:
for pk,p in bbTh.particles.iteritems():
    bbTh.positions[pk] = nwchem_i.strucC.positions[pk]
    print pk,p.symbol,bbTh.positions[pk]

Store the results in a tar ball in the storage directory 

In [ ]:
nwchem_i.store()

Now let us calculate the ESP charges to use in our forcefield 

Again let's make it a function

In [ ]:
def nw_esp(project_i,bb_i,res_i):
    '''Calculate ESP charges of a streamm Buildingblock object with nwchem 
    
    '''
    nwchem_esp = streamm.NWChem('nw_esp_{}'.format(bb_i.tag))
    print(nwchem_esp.tag)
    # Add thiophene structure with optimized coordinates from previous calculation
    nwchem_esp.strucC = copy.deepcopy(bb_i)
    # Set calculation to run on external resource
    nwchem_esp.set_resource(res_i)
    # Add calculation to project
    project_i.add_calc(nwchem_esp)
    # Make the local directories 
    nwchem_esp.make_dir()
    # Change to the `launch` directory
    os.chdir(nwchem_esp.dir['launch'])
    #
    nwchem_esp.cp_file('templates','run',"nwchem_peregrine.pbs",'templates','launch')
    nwchem_esp.cp_file('templates','nw',"nwchem_esp.nw",'templates','launch')
    #
    nwchem_esp.load_str('templates','nw')        
    nwchem_esp.load_str('templates','run')
    # 
    nwchem_esp.properties['basis'] = '6-31g'
    nwchem_esp.properties['method'] = 'UHF'
    nwchem_esp.properties['charge'] = 0
    nwchem_esp.properties['spin_mult'] = 1
    nwchem_esp.properties['task'] = 'SCF'
    nwchem_esp.properties['coord'] = nwchem_esp.strucC.write_coord()

    pprint(nwchem_esp.properties)

    nwchem_esp.replacewrite_prop('nw','input','nw','%s.nw'%(nwchem_esp.tag))

    nwchem_esp.properties['input_nw'] = nwchem_esp.files['input']['nw']
    nwchem_esp.replacewrite_prop('run','scripts','run','%s.pbs'%(nwchem_esp.tag))

    nwchem_esp.add_file('output','log',"%s.log"%(nwchem_esp.tag))

    # Save details in .json files 

    os.chdir(nwchem_esp.dir['home'])
    project_i.dump_json()

    os.chdir(nwchem_esp.dir['launch'])
    nwchem_esp.push()
    nwchem_esp.run()
    # Add calculation to project
    project_i.add_calc(nwchem_esp,deepcopy = True)
    # 
    return project_i 
    
    

In [ ]:
p3ht_et = nw_esp(p3ht_et,bbTh,peregrine)

Check status until finished

In [ ]:
nwchem_i = p3ht_et.calculations['nw_esp_thiophene']

In [ ]:
while( nwchem_i.meta['status'] != 'finished'):
    nwchem_i.check()
    time.sleep(30)    

In [ ]:
p3ht_et.check()

In [ ]:
nwchem_i.analysis()

In [ ]:
total_charge = 0.0 
for pk,p in nwchem_i.strucC.particles.iteritems():
    print pk,p.symbol,p.charge
    total_charge += p.charge
print total_charge

Update the charges of the Buildingblock

In [ ]:
bbTh.tag += '_HFesp'

In [ ]:
for pk,p in bbTh.particles.iteritems():
    p.charge = nwchem_i.strucC.particles[pk].charge
    print pk,p.symbol,p.charge

Store the results 

In [ ]:
nwchem_i.store()

In [ ]:
bbTh.bonded_nblist = bbTh.guess_nblist(0,radii_buffer=1.35)

In [ ]:
bbTh.bonded_bonds()
bbTh.bonded_angles()
bbTh.bonded_dih()

Store a pickled object of the Buildingblock

In [ ]:
os.chdir(res_local.dir['materials']) 
bbTh.dump_pickle()

Let us optimize the structure with the oplsaa force-field to check the parameters 

In [ ]:
os.chdir(res_local.dir['home']) 

In [ ]:
from pathlib2 import Path

In [ ]:
need_files = ['oplsaa.pkl']
for f in need_files:
    path = Path(f)
    if not path.is_file():
        print("Need to run forcefields.ipynb")
        os.system("jupyter nbconvert --to python  forcefields.ipynb")
        os.system("python forcefields.py")

In [ ]:
oplsaa = streamm.forcefields.parameters.read_pickle('oplsaa')

In [ ]:
print oplsaa.unit_conf['energy']

We need to add the conjugated carbons, hydrogen and sulfur atom types 

In [ ]:
import streamm.forcefields.particletype as particletype

In [ ]:
import pymatgen_core.core.periodic_table as periodic_table

Set some parameters from J. Am. Chem. Soc., 1996, 118 (45), pp 11225–11236

In [ ]:
CA = particletype.Particletype('CA')
HA = particletype.Particletype('HA')

In [ ]:
CA.update_units(oplsaa.unit_conf)
HA.update_units(oplsaa.unit_conf)

In [ ]:
CA.epsilon = 0.070 # kcal/mol
CA.sigma = 3.55 # Angstroms 

In [ ]:
HA.epsilon = 0.030 # kcal/mol
HA.sigma = 2.42 # Angstroms 

In [ ]:
CA.mass =  periodic_table.Element['C'].atomic_mass.real
HA.mass =  periodic_table.Element['H'].atomic_mass.real

In [ ]:
print CA,HA

In [ ]:
S = particletype.Particletype('S')

In [ ]:
S.update_units(oplsaa.unit_conf)

Set some parameters from J. Am. Chem. Soc., 1996, 118 (45), pp 11225–11236

In [ ]:
S.epsilon = 0.25 # kcal/mol
S.sigma = 3.55 # Angstroms 

In [ ]:
S.mass =  periodic_table.Element['S'].atomic_mass.real

Add to forcefield parameters container

In [ ]:
oplsaa.add_particletype(CA)
oplsaa.add_particletype(HA)
oplsaa.add_particletype(S)

Set the bond stretching parameters 

In [ ]:
import streamm.forcefields.bondtype as bondtype

In [ ]:
bt_i = bondtype.Bondtype('CA','HA',unit_conf=oplsaa.unit_conf)
bt_i.setharmonic(1.080,367.0)
oplsaa.add_bondtype(bt_i)

In [ ]:
bt_i = bondtype.Bondtype('CA','CA',unit_conf=oplsaa.unit_conf)
bt_i.setharmonic(1.400,469.0)
oplsaa.add_bondtype(bt_i)

In [ ]:
bt_i = bondtype.Bondtype('S','CA',unit_conf=oplsaa.unit_conf)
bt_i.setharmonic(1.71,250.0)
oplsaa.add_bondtype(bt_i)

In [ ]:
for btk,bt in oplsaa.bondtypes.iteritems():
    print btk,bt

In [ ]:
import streamm.forcefields.angletype as angletype

In [ ]:
bat_i = angletype.Angletype('CA','CA','CA',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(120.0,63.0)
oplsaa.add_angletype(bat_i)

In [ ]:
bat_i = angletype.Angletype('CA','CA','HA',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(120.0,35.0)
oplsaa.add_angletype(bat_i)

In [ ]:
bat_i = angletype.Angletype('CA','S','CA',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(92.2,70.0)
oplsaa.add_angletype(bat_i)

In [ ]:
bat_i = angletype.Angletype('S','CA','HA',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(120.0,35.0)
oplsaa.add_angletype(bat_i)

In [ ]:
bat_i = angletype.Angletype('S','CA','CA',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(111.0,70.0)
oplsaa.add_angletype(bat_i)

In [ ]:
for atk,at in oplsaa.angletypes.iteritems():
    print atk,at

Set some reasonable dihedral parameters

In [ ]:
import streamm.forcefields.dihtype as dihtype

In [ ]:
# oplsaa.dihtypes = {}

In [ ]:
dih_i = dihtype.Dihtype('X','CA','CA','X',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,1.812532,0.0,0.0)
oplsaa.add_dihtype(dih_i)

In [ ]:
dih_i = dihtype.Dihtype('X','S','CA','X',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,2.416710,0.0,0.0)
oplsaa.add_dihtype(dih_i)

In [ ]:
dih_i = dihtype.Dihtype('S','CA','CA','HA',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,1.812532,0.0,0.0)
oplsaa.add_dihtype(dih_i)

In [ ]:
for dk,d in oplsaa.dihtypes.iteritems():
    print dk,d 

Let us make an MD simulation of just the monomer to check that our parameters are okay

In [ ]:
def lmp_run(project_i,bb_i,param_i,res_i,md_type = 'min'):
    # Create LAMMPS calculation object 
    lmp_i = streamm.LAMMPS('lmp_{}_{}'.format(md_type,bb_i.tag))
    # Set parameter container 
    lmp_i.paramC = param_i
    lmp_i.set_strucC(bb_i)
    # Set force-field parameters 
    lmp_i.set_ffparam()
    # Set resource to local
    lmp_i.set_resource(res_i)
    # Make local directories
    lmp_i.make_dir()
    # Set pbc's to on
    lmp_i.strucC.lat.pbcs = [True,True,True]
    # Change to launch directory
    os.chdir(lmp_i.dir['launch'])
    # Copy over the templates from the template directory 
    lmp_i.cp_file('templates','in',"lammps_{}.in".format(md_type),'templates','launch')
    lmp_i.cp_file('templates','run',"lammps_peregrine.pbs",'templates','launch')
    # Change to scratch
    os.chdir(lmp_i.dir['launch'])
    # Read in template files and store them as strings in the `str` dictionary
    lmp_i.load_str('templates','in')
    lmp_i.load_str('templates','run')
    # Write LAMMPS .data file
    lmp_i.write_data()
    # Replace keys in template string with properties 
    lmp_i.replacewrite_prop('in','input','in','%s.in'%(lmp_i.tag))
    # Add the input file to the properties to be written into the run file
    lmp_i.properties['input_in'] = lmp_i.files['input']['in']
    lmp_i.replacewrite_prop('run','scripts','run','%s.pbs'%(lmp_i.tag))
    # Save json file in root directory
    os.chdir(lmp_i.dir['home'])
    lmp_i.dump_json()
    # Run bash script or submit to cluster
    lmp_i.add_file('output','log',"%s.log"%(lmp_i.tag))
    # Save details in .json files 
    os.chdir(lmp_i.dir['home'])
    project_i.dump_json()
    #
    os.chdir(lmp_i.dir['launch'])
    lmp_i.push()
    lmp_i.run()
    # Add calculation to project
    project_i.add_calc(lmp_i,deepcopy = True)
    # 
    return project_i     

In [ ]:
p3ht_et = lmp_run(p3ht_et,bbTh,oplsaa,peregrine)

In [ ]:
lmp_i = p3ht_et.calculations['lmp_min_thiophene']

In [ ]:
while( lmp_i.meta['status'] != 'finished'):
    lmp_i.check()
    time.sleep(30)

In [ ]:
lmp_i.analysis()

Energy decreased and nothing exploded so that's good

In [ ]:
lmp_i.store()

Read in data file positions

In [ ]:
lmp_i.pull()

Read in data file output and update positions

In [ ]:
print lmp_i.dir['launch']
os.chdir(lmp_i.dir['launch'])

In [ ]:
datafn = lmp_i.files['output']['data_1']
print datafn

In [ ]:
lmp_i.read_data_pos(datafn)

In [ ]:
print lmp_i.strucC.lat.matrix

In [ ]:
lmp_i.strucC.write_xyz()

We will use the oplsaa optimized structure as the initial structure since we will be running MD 

In [ ]:
bbTh.tag += '_oplsaa'

In [ ]:
for pk,p in bbTh.particles.iteritems():
    bbTh.positions[pk] = lmp_i.strucC.positions[pk]
    print pk,p.symbol,bbTh.positions[pk]

Save the Buildingblock and force-field

In [ ]:
os.chdir(res_local.dir['materials']) 
bbTh.write_xyz()
# bbTh.dump_pickle() # Not working for 
oplsaa.dump_pickle()

Okay now that we have a handle on thiophene let's follow the same procedure for hexane

Build hexane

In [ ]:
bbHex = streamm.Buildingblock('hexane')
symbols = ['C','H','H','H','C','H','H','C','H','H','C','H','H','C','H','H','C','H','H','H']
positions = [ ]
positions.append([-6.410969,-0.381641,-0.000031])
positions.append([-7.310084,0.245311,-0.000038])
positions.append([-6.456117,-1.028799,0.884636])
positions.append([-6.456111,-1.028812,-0.884689])
positions.append([-5.135268,0.467175,-0.000033])
positions.append([-5.135484,1.128782,0.877977])
positions.append([-5.135479,1.128771,-0.87805])
positions.append([-3.850566,-0.371258,-0.000024])
positions.append([-3.85112,-1.033978,0.87841])
positions.append([-3.851114,-1.033987,-0.878451])
positions.append([-2.567451,0.469603,-0.000024])
positions.append([-2.567784,1.132155,0.8784])
positions.append([-2.567776,1.132146,-0.878455])
positions.append([-1.283527,-0.370234,-0.000013])
positions.append([-1.28337,-1.032804,0.87836])
positions.append([-1.28336,-1.032812,-0.87838])
positions.append([0.00482234,0.47342231,-0.00000898])
positions.append([0.02595107,1.09220686,0.87266464])
positions.append([0.85585781,-0.17514133,0.00194589])
positions.append([0.02780957,1.08937798,-0.87463473])
for i in range(len(symbols)):
    pt_i = streamm.Particle(symbol=symbols[i])
    pos_i = positions[i]
    bbHex.add_partpos(pt_i,pos_i)

In [ ]:
bbHex.particles[0].rsite = 'rg'
bbHex.particles[1].rsite = 'rgcap'

In [ ]:
c_cnt =1
h_cnt =1
for pkey_i, particle_i  in bbHex.particles.iteritems():
            if( particle_i.symbol == 'C' ):
                particle_i.label = "C%d"%(c_cnt)
                particle_i.resname = "SCP3"
                particle_i.residue = c_cnt
                c_cnt +=1 
            if( particle_i.symbol == 'H' ):
                particle_i.label = "H%d"%(h_cnt)
                particle_i.resname = "HC"
                particle_i.residue = c_cnt -1 
                h_cnt +=1 

Set the parameter keys and some reasonable atomic charges 

In [ ]:
for pkey_i, particle_i  in bbHex.particles.iteritems():
            if( particle_i.symbol == 'C' ):
                particle_i.paramkey = 'CT'
                particle_i.charge = -0.12

            if( particle_i.symbol == 'H' ):
                particle_i.paramkey = 'HC'
                particle_i.charge = 0.06
            print pkey_i, particle_i.symbol,particle_i.charge

In [ ]:
bbHex.particles[0].charge  = -0.18
bbHex.particles[16].charge  = -0.18

Check that the molecule is neutral 

In [ ]:
bbHex.calc_charge()
print bbHex.charge


Now let us optimze and calculate ESP charges for hexane

Optimize structure with NWChem

In [ ]:
p3ht_et = nw_opt(p3ht_et,bbHex,peregrine)

In [ ]:
nwchem_i = p3ht_et.calculations['nw_opt_hexane']

In [ ]:
while( nwchem_i.meta['status'] != 'finished'):
    nwchem_i.check()
    time.sleep(30)

In [ ]:
p3ht_et.check()

Get the calculation from the project object 

In [ ]:
nwchem_i.analysis()

Print energies 

In [ ]:
print nwchem_i.properties['alpha_energies'][10:20]
print nwchem_i.properties['energy']

Check that the positions of the structure have been optimized 

In [ ]:
for pk,p in bbHex.particles.iteritems():
    print pk,p.symbol,bbHex.positions[pk]

In [ ]:
print nwchem_i.strucC.positions

Update positions in Buildingblock object

In [ ]:
for pk,p in bbHex.particles.iteritems():
    bbHex.positions[pk] = nwchem_i.strucC.positions[pk]
    print pk,p.symbol,bbHex.positions[pk]

Store the results in a tar ball in the storage directory 

In [ ]:
nwchem_i.store()

Now let us calculate the ESP charges to use in our forcefield 

In [ ]:
p3ht_et = nw_esp(p3ht_et,bbHex,peregrine)

Check status unit finished

In [ ]:
nwchem_i = p3ht_et.calculations['nw_esp_hexane']

In [ ]:
while( nwchem_i.meta['status'] != 'finished'):
    nwchem_i.check()
    time.sleep(30)

In [ ]:
p3ht_et.check()

In [ ]:
nwchem_i.analysis()

In [ ]:
for pk,p in nwchem_i.strucC.particles.iteritems():
    print pk,p.symbol,p.charge

In [ ]:
nwchem_i.strucC.calc_charge()
print nwchem_i.strucC.charge

Print energies 

In [ ]:
print nwchem_i.properties['alpha_energies'][10:20]
print nwchem_i.properties['energy']

Update the charges of the Buildingblock

In [ ]:
for pk,p in bbHex.particles.iteritems():
    p.charge = nwchem_i.strucC.particles[pk].charge

In [ ]:
bbHex.tag += '_HFesp'

Store the results 

In [ ]:
nwchem_i.store()

First we need to identify the bonding within the Buildingblock

In [ ]:
bbHex.bonded_nblist = bbHex.guess_nblist(0,radii_buffer=1.35)

In [ ]:
bbHex.bonded_bonds()
bbHex.bonded_angles()
bbHex.bonded_dih()

Add the need parameters the the oplsaa parameter container

In [ ]:
bat_i = angletype.Angletype('CT','CT','CT',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(109.50,40.0)
oplsaa.add_angletype(bat_i)

In [ ]:
bat_i = angletype.Angletype('CT','CT','CT',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(109.50,40.0)
oplsaa.add_angletype(bat_i)

In [ ]:
bat_i = angletype.Angletype('CT','CT','HC',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(109.50,50.0)
oplsaa.add_angletype(bat_i)

In [ ]:
dih_i = dihtype.Dihtype('CT','CT','CT','CT',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.433341,-0.016667,0.066668,0.0)
oplsaa.add_dihtype(dih_i)

In [ ]:
dih_i = dihtype.Dihtype('HC','CT','CT','CT',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,-0.0,0.1,0.0)
oplsaa.add_dihtype(dih_i)

In [ ]:
dih_i = dihtype.Dihtype('HC','CT','CT','HC',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,-0.0,0.1,0.0)
oplsaa.add_dihtype(dih_i)

Run a oplsaa minimization to get the minimized structure

In [ ]:
p3ht_et = lmp_run(p3ht_et,bbHex,oplsaa,peregrine)

In [ ]:
p3ht_et.check()

In [ ]:
lmp_i = p3ht_et.calculations['lmp_min_hexane_HFesp']

In [ ]:
while( lmp_i.meta['status'] != 'finished'):
    lmp_i.check()
    time.sleep(30)

In [ ]:
lmp_i.analysis()

Energy decreased and nothing exploded so that's good

In [ ]:
lmp_i.store()

Read in data file positions

In [ ]:
lmp_i.pull()

Read in data file output and update positions

In [ ]:
print lmp_i.dir['launch']
os.chdir(lmp_i.dir['launch'])

In [ ]:
datafn = lmp_i.files['output']['data_1']
print datafn

In [ ]:
lmp_i.read_data_pos(datafn)

In [ ]:
print lmp_i.strucC.lat.matrix

In [ ]:
lmp_i.strucC.write_xyz()

We will use the oplsaa optimized structure as the initial structure since we will be running MD 

In [ ]:
bbHex.tag += '_oplsaa'

In [ ]:
for pk,p in bbHex.particles.iteritems():
    bbHex.positions[pk] = lmp_i.strucC.positions[pk]
    print pk,p.symbol,bbHex.positions[pk]

Save the Buildingblock and force-field

In [ ]:
os.chdir(res_local.dir['materials']) 
bbHex.write_xyz()
# bbTh.dump_pickle() # Not working for 
oplsaa.dump_pickle()

In [ ]:
print bbHex.tag,bbTh.tag

So let us make some P3HT oligomers 

In [ ]:
os.chdir(res_local.dir['materials']) 

In [ ]:
bbTh.find_rsites()
bbHex.find_rsites()

In [ ]:
print(bbTh.show_rsites())

In [ ]:
print(bbHex.show_rsites())

In [ ]:
import streamm.structures.buildingblock as bb

In [ ]:
ht = bb.attach(bbTh,bbHex,'funccap',0,'rgcap',0,tag='3-hexyl-thiophene')

Update bond angles and dihedrals after Buildingblock join

In [ ]:
ht.bonded_bonds()
ht.bonded_angles()
ht.bonded_dih()

Check that the molecule looks good

In [ ]:
ht.write_xyz()

Check the charges of the removed hydrogens got summed onto the functionalized carbons correctly

In [ ]:
ht.calc_charge()
ht.charge

In [ ]:
print(ht.show_rsites())

Add inter thiophene hexane parameters

In [ ]:
bt_i = bondtype.Bondtype('CT','CA',unit_conf=oplsaa.unit_conf)
bt_i.setharmonic(1.51,317.0)
oplsaa.add_bondtype(bt_i)

Bond angle parameters 

In [ ]:
bat_i = angletype.Angletype('CA','CA','CT',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(120.0,70.0)
oplsaa.add_angletype(bat_i)


bat_i = angletype.Angletype('HA','CA','CT',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(120.0,35.0)
oplsaa.add_angletype(bat_i)



bat_i = angletype.Angletype('CA','CT','HC',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(109.5,50.0)
oplsaa.add_angletype(bat_i)

bat_i = angletype.Angletype('CA','CT','CT',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(114.0,63.0)
oplsaa.add_angletype(bat_i)

In [ ]:
for atk,at in oplsaa.angletypes.iteritems():
    print atk,at

Note: The inter-ring torsional is not consider as a seperate set of parameters for the simplicity of this example

In [ ]:
dih_i = dihtype.Dihtype('HC','CT','CT','CA',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,-0.0,0.1,0.0)
oplsaa.add_dihtype(dih_i)

In [ ]:
dih_i = dihtype.Dihtype('CT','CT','CT','CA',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.433341,-0.016667,0.066668,0.0)
oplsaa.add_dihtype(dih_i)

In [ ]:
dih_i = dihtype.Dihtype('HC','CT','CA','CA',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,-0.0,0.1,0.0)
oplsaa.add_dihtype(dih_i)

In [ ]:
dih_i = dihtype.Dihtype('CT','CT','CA','CA',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,-0.0,0.0,0.0)
oplsaa.add_dihtype(dih_i)

In [ ]:
for dk,d in oplsaa.dihtypes.iteritems():
    print dk,d 

Run a oplsaa minimization to get the minimized structure

In [ ]:
p3ht_et = lmp_run(p3ht_et,ht,oplsaa,peregrine)

In [ ]:
p3ht_et.check()

In [ ]:
lmp_i = p3ht_et.calculations['lmp_min_3-hexyl-thiophene']

In [ ]:
while( lmp_i.meta['status'] != 'finished'):
    lmp_i.check()
    time.sleep(30)

In [ ]:
lmp_i.analysis()

Energy decreased and nothing exploded so that's good

In [ ]:
lmp_i.store()

Read in data file positions

In [ ]:
lmp_i.pull()

Read in data file output and update positions

In [ ]:
print lmp_i.dir['launch']
os.chdir(lmp_i.dir['launch'])

In [ ]:
datafn = lmp_i.files['output']['data_1']
print datafn

In [ ]:
lmp_i.read_data_pos(datafn)

In [ ]:
print lmp_i.strucC.lat.matrix

In [ ]:
lmp_i.strucC.write_xyz()

We will use the oplsaa optimized structure as the initial structure since we will be running MD 

In [ ]:
ht.tag += '_oplsaa'

In [ ]:
for pk,p in ht.particles.iteritems():
    ht.positions[pk] = lmp_i.strucC.positions[pk]
    print pk,p.symbol,ht.positions[pk]

Save the Buildingblock and force-field

In [ ]:
os.chdir(res_local.dir['materials']) 
ht.write_xyz()
# bbTh.dump_pickle() # Not working for 
oplsaa.dump_pickle()

Okay we have the monomer, so let's make a pentamer 

In [ ]:
penta_ht = copy.deepcopy(ht)

In [ ]:
# penta_ht = ht.prepattach('termcap',0,dir=-1,yangle=90.0)

In [ ]:
for n in range(4):
    penta_ht = bb.attach(penta_ht,ht,'termcap',1,'termcap',0,tag='penta_3-hexyl-thiophene')

Check the charges of the removed hydrogens got summed onto the functionalized carbons correctly

In [ ]:
penta_ht.calc_charge()
penta_ht.charge

In [ ]:
penta_ht.write_xyz()

Well it's cis, but we can run some high temperature MD to randomize that 

Update bond angles and dihedrals after Buildingblock join

In [ ]:
penta_ht.bonded_bonds()
penta_ht.bonded_angles()
penta_ht.bonded_dih()

In [ ]:
print penta_ht.print_properties()

Run a oplsaa minimization to get the minimized structure

In [ ]:
p3ht_et = lmp_run(p3ht_et,penta_ht,oplsaa,peregrine)

In [ ]:
p3ht_et.check()

In [ ]:
lmp_i = p3ht_et.calculations['lmp_min_penta_3-hexyl-thiophene']

In [ ]:
while( lmp_i.meta['status'] != 'finished'):
    lmp_i.check()
    # time.sleep(30)

In [ ]:
lmp_i.analysis()

Energy decreased and nothing exploded so that's good

In [ ]:
lmp_i.store()

Read in data file positions

In [ ]:
lmp_i.pull()

Read in data file output and update positions

In [ ]:
print lmp_i.dir['launch']
os.chdir(lmp_i.dir['launch'])

In [ ]:
datafn = lmp_i.files['output']['data_1']
print datafn

In [ ]:
lmp_i.read_data_pos(datafn)

In [ ]:
print lmp_i.strucC.lat.matrix

In [ ]:
lmp_i.strucC.write_xyz()

We will use the oplsaa optimized structure as the initial structure since we will be running MD 

In [ ]:
penta_ht.tag += '_oplsaa'

In [ ]:
for pk,p in penta_ht.particles.iteritems():
    penta_ht.positions[pk] = lmp_i.strucC.positions[pk]
    print pk,p.symbol,penta_ht.positions[pk]

Save the Buildingblock and force-field

In [ ]:
os.chdir(res_local.dir['materials']) 
penta_ht.write_xyz()
# bbTh.dump_pickle() # Not working for 
oplsaa.dump_pickle()

Cool let's run some MD

In [ ]:
p3ht_et = lmp_run(p3ht_et,penta_ht,oplsaa,peregrine,md_type='nvt')

In [ ]:
p3ht_et.check()

In [ ]:
lmp_i = p3ht_et.calculations['lmp_min_3-hexyl-thiophene']

In [ ]:
while( lmp_i.meta['status'] != 'finished'):
    lmp_i.check()
    time.sleep(30)

In [ ]:
lmp_i.analysis()

Sweet as bro!